In [1]:
%load_ext autoreload
# %reload_ext autoreload

# Reload all modules imported with %aimport every time before executing the Python code typed
%autoreload 1

%aimport context_nn 
%aimport phrase_feeder 
%aimport bitnotes
%aimport watch_point
%aimport cluster
%aimport constants
import numpy as np
from bitarray import bitarray
from context_nn import ContextNN
from watch_point import WatchPoint
from cluster import Cluster
from phrase_feeder import PhraseFeeder
from bitnotes import BitNotes
from pprint import pprint
import math
import constants as const

In [2]:
import pickle

def load_cxnn(file_name: str) -> ContextNN:
    with open(file_name, 'rb') as f:
        cxnn = pickle.load(f)
        return cxnn
    
def load_noted_phrases(file_name: str) -> list:
    with open(file_name, 'rb') as f:
        noted_phrases = pickle.load(f)
        return noted_phrases

In [49]:
cxnn = load_cxnn('./data/cxnn_langs.pickle')
noted_phrases = load_noted_phrases('./data/texts/tst_rus.pickle')

In [ ]:
# cxnn = load_cxnn('./data/cxnn_monotone.pickle')
# noted_phrases = load_noted_phrases('./data/texts/tst_bbb.pickle')

In [28]:
cxnn = load_cxnn('./data/cxnn_probs.pickle')
noted_phrases = load_noted_phrases('./data/texts/tst_3331.pickle')

In [61]:
cxnn = load_cxnn('./data/cxnn_dna.pickle')
noted_phrases = load_noted_phrases('./data/texts/tst_xyzw.pickle')

In [62]:
%%time

cxnn.start_detection()

for num, phrase in enumerate(noted_phrases):
    bit_chord = cxnn.bit_notes.phrase_chord_as_bits(phrase)
    cxnn.detect_bits(bit_chord)
    if num % 500 == 0:
        print(f'phrase # {num}/{len(noted_phrases)}')

phrase # 0/4975
phrase # 500/4975
phrase # 1000/4975
phrase # 1500/4975
phrase # 2000/4975
phrase # 2500/4975
phrase # 3000/4975
phrase # 3500/4975
phrase # 4000/4975
phrase # 4500/4975
Wall time: 2min 22s


In [ ]:
print(cxnn.cluster_count())
pprint(cxnn.cluster_len_stats())
pprint(cxnn.cluster_activity_stats())

In [58]:
cxnn.data_marks

{(0,): 'abcd',
 (1,): 'cdef',
 (2,): 'klmn',
 (3,): 'ijhg',
 (4,): 'oprs',
 (5,): 'xyzw',
 (6,): 'quva',
 (7,): 'kiox',
 (8,): 'nghp'}

In [63]:
detection = cxnn.summarize_detection()

0 18.298720467028673
1 20.753455415591816
2 8.047262232465112
3 22.00655917664805
4 15.782150939144234
5 1857.7480487590142
6 12.095660126044427
7 257.63353864402916
8 18.701310758576522


In [65]:
for bits, label in cxnn.data_marks.items():
    rate = np.sum(detection[list(bits)])
    print(label, rate)

abcd 18.298720467
cdef 20.7534554156
klmn 8.04726223247
ijhg 22.0065591766
oprs 15.7821509391
xyzw 1857.74804876
quva 12.095660126
kiox 257.633538644
nghp 18.7013107586


In [30]:
cxnn.vectors_received

4975

In [64]:
print(cxnn.cluster_count())
points = [(bit, count) for bit, count in cxnn.point_stats() if count > 0]
# print(sorted(points, key=lambda x: x[0]))
bit_points = {}
for bit, count in points:
    bit_points[bit] = bit_points.get(bit, 0) + 1

print(sorted(bit_points.items()))
    
bit_clusters = {}
for bit, count in points:
    bit_clusters[bit] = bit_clusters.get(bit, 0) + count

print(sorted(bit_clusters.items()))

bit_activations = {}
for point in cxnn.point_objects:
    key = point.output_bit
    bit_activations[key] = bit_activations.get(key, 0) + sum(cluster.activation_count() 
                                                             for cluster in point.cluster_objects 
                                                             if cluster.consolidations >= 0)

print(sorted(bit_activations.items()))

bit_divs = {}
for key in bit_clusters.keys():
    bit_divs[key] = bit_activations[key] / (bit_clusters[key] * bit_points[key])

pprint(sorted(bit_divs.items()))


52241
[(0, 176), (1, 154), (2, 164), (3, 159), (4, 163), (5, 159), (6, 154), (7, 153), (8, 160)]
[(0, 6195), (1, 6100), (2, 5205), (3, 5867), (4, 5477), (5, 6492), (6, 5850), (7, 5504), (8, 5551)]
[(0, 49576), (1, 48145), (2, 16025), (3, 73715), (4, 38470), (5, 5600855), (6, 36042), (7, 803041), (8, 61781)]
[(0, 0.045469220045491233),
 (1, 0.05125079838194592),
 (2, 0.01877299032356318),
 (3, 0.07902102474880822),
 (4, 0.04309152272022098),
 (5, 5.425986313101369),
 (6, 0.040006660006660005),
 (7, 0.9536035586715307),
 (8, 0.06956066474509097)]


In [11]:
sum([624, 342637, 0.0, 351069, 2021, 789, 350828, 2955, 274, 1810, 337907, 1010, 3099])

1395023.0

In [13]:
sum([556, 593, 1233, 3074, 4708, 689398, 24, 10971, 4258, 492475, 2502, 6425])

1216217

In [ ]:
[0.0, 0.0, 596, 268, 520, 81307, 0.0, 0.0, 0.0, 0.0, 310, 0.0, 79]
[0.0, 0.0, 678, 189, 268, 111389, 0.0, 0.0, 0.0, 0.0, 158, 0.0, 50]